Predicting Runs Scored: Data Wrangling

Last notebook we decided to switch our focus onto predicting runs scored
This could be useful in several capacities:

    - For predicting the over/under

    - To create features for predicting the game winner


We will focus on predicting the distribution of runs scored - that is, putting a probability on each possible value of runs scored for a team (up to some maximum value)

Predicting the distribution of a numeric target variable is known as "probabilistic regression"

In this notebook, we will "wrangle" our data such that each row represents a hitting team against a pitching team
The goal will be to predict the (distribution of the) number of runs scored by the hitting team

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [11]:
df = pd.read_csv('df_bp9.csv', low_memory=False)

Create df for runs of a single team

    - Want to "split" each game into two rows:

        - One where the home team bats against the visiting pitchers (and we try to predict the runs of the home team)

        - Other where we switch the roles of home and visiting

        - Then we can model how many runs we expect based on the hitters for that team and the opposing pitching

In [12]:
cols_ref = ['season', 'date','dblheader_code','date_dblhead','team_h','team_v',
            'league_h','league_v','over_under_line','over_under_result',
           'ballpark_id', 'day_night']

target_cols = ['runs_h', 'runs_v']

In [13]:
team_hit_stems = ['BATAVG','OBP','SLG','OBS','ERR','SB','CS']
lineup_hit_stems = ['BATAVG','OBP','SLG','OBS','SLGmod','SObat_perc']
strt_pitch_stems = ['ERA','WHIP','SO_perc','H_BB_perc','TB_BB_perc','FIP','FIP_perc']
bpen_pitch_stems = ['WHIP','SO_perc','H_BB_perc','TB_BB_perc']

team_hit_winsizes = [30,162]
lineup_hit_winsizes = [10,30,81,162]
strt_pitch_winsizes = [6,15,30]
bpen_pitch_winsizes = [10,30,162]

In [14]:
# We will make two data frames:
# version 'a': home hitting and visitor pitching
# version 'b': visitor hitting and home pitching

team_hit_features_a = [x+'_'+str(winsize)+'_h' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_a = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_h' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_a = ['Strt_'+x+'_'+str(winsize)+'_v' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_a = ['Bpen_'+x+'_'+str(winsize)+'_v' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

team_hit_features_b = [x+'_'+str(winsize)+'_v' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_b = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_v' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_b = ['Strt_'+x+'_'+str(winsize)+'_h' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_b = ['Bpen_'+x+'_'+str(winsize)+'_h' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

In [15]:
cols_a = cols_ref + ['runs_h'] + team_hit_features_a + lineup_hit_features_a + start_pitch_features_a + bpen_pitch_features_a
df_a = df.loc[:,cols_a]
df_a['home_hitting'] = 1


cols_b = cols_ref + ['runs_v'] + team_hit_features_b + lineup_hit_features_b + start_pitch_features_b + bpen_pitch_features_b
df_b = df.loc[:,cols_b]
df_b['home_hitting'] = 0

stripped_feats  = [x[:-2] for x in team_hit_features_a + lineup_hit_features_a + 
                                              start_pitch_features_a + bpen_pitch_features_a]

final_col_list = cols_ref +['runs'] + stripped_feats+ ['home_hitting']

df_a.columns = final_col_list
df_b.columns = final_col_list
df_runs = pd.concat((df_a,df_b))

In [17]:
df_w_odds = df_runs[df_runs['season'] >= 2019]
df_w_odds.sample(10)

,season,date,dblheader_code,date_dblhead,team_h,team_v,league_h,league_v,over_under_line,over_under_result,ballpark_id,day_night,runs,BATAVG_30,OBP_30,SLG_30,OBS_30,ERR_30,SB_30,CS_30,BATAVG_162,OBP_162,SLG_162,OBS_162,ERR_162,SB_162,CS_162,lineup8_BATAVG_10,lineup9_BATAVG_10,lineup8_BATAVG_10_w,lineup9_BATAVG_10_w,lineup8_OBP_10,lineup9_OBP_10,lineup8_OBP_10_w,lineup9_OBP_10_w,lineup8_SLG_10,lineup9_SLG_10,lineup8_SLG_10_w,lineup9_SLG_10_w,lineup8_OBS_10,lineup9_OBS_10,lineup8_OBS_10_w,lineup9_OBS_10_w,lineup8_SLGmod_10,lineup9_SLGmod_10,lineup8_SLGmod_10_w,lineup9_SLGmod_10_w,lineup8_SObat_perc_10,lineup9_SObat_perc_10,lineup8_SObat_perc_10_w,lineup9_SObat_perc_10_w,lineup8_BATAVG_30,lineup9_BATAVG_30,lineup8_BATAVG_30_w,lineup9_BATAVG_30_w,lineup8_OBP_30,lineup9_OBP_30,lineup8_OBP_30_w,lineup9_OBP_30_w,lineup8_SLG_30,lineup9_SLG_30,lineup8_SLG_30_w,lineup9_SLG_30_w,lineup8_OBS_30,lineup9_OBS_30,lineup8_OBS_30_w,lineup9_OBS_30_w,lineup8_SLGmod_30,lineup9_SLGmod_30,lineup8_SLGmod_30_w,lineup9_SLGmod_30_w,lineup8_SObat_perc_30,lineup9_SObat_perc_30,lineup8_SObat_perc_30_w,lineup9_SObat_perc_30_w,lineup8_BATAVG_81,lineup9_BATAVG_81,lineup8_BATAVG_81_w,lineup9_BATAVG_81_w,lineup8_OBP_81,lineup9_OBP_81,lineup8_OBP_81_w,lineup9_OBP_81_w,lineup8_SLG_81,lineup9_SLG_81,lineup8_SLG_81_w,lineup9_SLG_81_w,lineup8_OBS_81,lineup9_OBS_81,lineup8_OBS_81_w,lineup9_OBS_81_w,lineup8_SLGmod_81,lineup9_SLGmod_81,lineup8_SLGmod_81_w,lineup9_SLGmod_81_w,lineup8_SObat_perc_81,lineup9_SObat_perc_81,lineup8_SObat_perc_81_w,lineup9_SObat_perc_81_w,lineup8_BATAVG_162,lineup9_BATAVG_162,lineup8_BATAVG_162_w,lineup9_BATAVG_162_w,lineup8_OBP_162,lineup9_OBP_162,lineup8_OBP_162_w,lineup9_OBP_162_w,lineup8_SLG_162,lineup9_SLG_162,lineup8_SLG_162_w,lineup9_SLG_162_w,lineup8_OBS_162,lineup9_OBS_162,lineup8_OBS_162_w,lineup9_OBS_162_w,lineup8_SLGmod_162,lineup9_SLGmod_162,lineup8_SLGmod_162_w,lineup9_SLGmod_162_w,lineup8_SObat_perc_162,lineup9_SObat_perc_162,lineup8_SObat_perc_162_w,lineup9_SObat_perc_162_w,Strt_ERA_6,Strt_WHIP_6,Strt_SO_perc_6,Strt_H_BB_perc_6,Strt_TB_BB_perc_6,Strt_FIP_6,Strt_FIP_perc_6,Strt_ERA_15,Strt_WHIP_15,Strt_SO_perc_15,Strt_H_BB_perc_15,Strt_TB_BB_perc_15,Strt_FIP_15,Strt_FIP_perc_15,Strt_ERA_30,Strt_WHIP_30,Strt_SO_perc_30,Strt_H_BB_perc_30,Strt_TB_BB_perc_30,Strt_FIP_30,Strt_FIP_perc_30,Bpen_WHIP_10,Bpen_SO_perc_10,Bpen_H_BB_perc_10,Bpen_TB_BB_perc_10,Bpen_WHIP_30,Bpen_SO_perc_30,Bpen_H_BB_perc_30,Bpen_TB_BB_perc_30,Bpen_WHIP_162,Bpen_SO_perc_162,Bpen_H_BB_perc_162,Bpen_TB_BB_perc_162,home_hitting
46433,2019,20190418,0,201904180,WAS,SFN,NL,NL,7.5,U,WAS11,D,2,0.207882,0.258303,0.325123,0.583426,18.0,11.0,7.0,0.236209,0.292867,0.365527,0.658394,100.0,74.0,33.0,0.197092,0.175193,0.200160,0.180745,0.275824,0.245177,0.278501,0.251488,0.350641,0.311681,0.358428,0.323662,0.626465,0.556858,0.636929,0.575151,0.484098,0.446976,0.490751,0.457700,0.246961,0.236188,0.249363,0.239725,0.207852,0.184757,0.209440,0.189125,0.269132,0.239228,0.270145,0.243942,0.355010,0.315565,0.360147,0.325215,0.624142,0.554793,0.630292,0.569157,0.454773,0.411650,0.461032,0.422781,0.241667,0.222223,0.245134,0.227824,0.224656,0.204495,0.225409,0.207737,0.279113,0.252901,0.279780,0.256834,0.366940,0.334400,0.370207,0.341484,0.646053,0.587301,0.649988,0.598318,0.428870,0.405909,0.434717,0.414106,0.226364,0.222475,0.229920,0.226180,0.227532,0.206366,0.227126,0.208688,0.281879,0.255017,0.281486,0.258075,0.384290,0.348450,0.384853,0.353511,0.666169,0.603467,0.666338,0.611586,0.437775,0.407309,0.440577,0.413710,0.222112,0.212522,0.224086,0.215523,3.545455,1.212121,0.294118,0.294118,0.441176,3.181818,0.772059,3.111524,1.070632,0.311111,0.266667,0.394444,1.873606,0.466667,3.318436,1.122905,0.296143,0.276860,0.393939,1.983240,0.488981,1.588235,0.213740,0.343511,0.587786,1.574257,0.215078,0.352550,0.525499,1.319771,0.219780,0.316850,0.482143,0
50692,2021,20210701,0,202107010,OAK,TEX,AL,AL,8.0,O,OAK01,D,8,0.230539,0.291360,0.375250,0.666610,13.0,14.0,7.0,0.231286,0.294501,0.382

In [18]:
df_runs.to_csv('df_runs_bp11.csv', index=False)